In [120]:
import pandas as pd
import numpy as np
import pydeck as pdk
import random
import shapefile
import json
import math

import geopy.distance

from tqdm import tqdm

from multiprocessing import Pool
from IPython.display import clear_output

In [2]:
def read_shapefile(sf_shape):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' 
    column holding the geometry information. This uses the pyshp
    package
    """

    fields = [x[0] for x in sf_shape.fields][1:]
    records = [y[:] for y in sf_shape.records()]
    #records = sf_shape.records()
    shps = [s.points for s in sf_shape.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df


In [3]:
#загружаем информацию о сеточном разбиении МО в память
myshp = open("fishnet2021.shp", "rb")
mydbf = open("fishnet2021.dbf", "rb")
r = shapefile.Reader(shp=myshp, dbf=mydbf)
s_df = read_shapefile(r)
#в поле coords первая координата центр квадрата, остальные его углы
s_df.head()

,cell_zid,coords
0,0,"[(38.64720691553791, 54.255243332408604), (38...."
1,1,"[(38.65487820205851, 54.255170084754035), (38...."
2,2,"[(38.66254945426432, 54.255096350709195), (38...."
3,3,"[(38.670220672771414, 54.25502212698049), (38...."
4,4,"[(38.67789185552521, 54.25494741523556), (38.6..."


In [32]:
#считываем информацию о переходах Дом-Работа и прикручиваем туда координаты переходов
h_w_matrix = pd.read_csv("04_CMatrix_Home_Work_July.csv")
h_w_matrix = h_w_matrix.merge(right=s_df, how='inner', left_on='home_zid', right_on='cell_zid')
h_w_matrix = h_w_matrix.merge(right=s_df, how='inner', left_on='work_zid', right_on='cell_zid')
h_w_matrix.drop('cell_zid_x', axis=1, inplace=True)
h_w_matrix.drop('cell_zid_y', axis=1, inplace=True)

h_w_matrix.head()

,home_zid,work_zid,customers_cnt,coords_x,coords_y
0,33617,35383,3,"[(37.04909381713915, 55.14932918598869), (37.0...","[(37.13516508529589, 55.172076772159485), (37...."
1,33620,33621,2,"[(37.07262354156165, 55.149413945205254), (37....","[(37.08046681019148, 55.14944118799408), (37.0..."
2,33621,33621,2,"[(37.08046681019148, 55.14944118799408), (37.0...","[(37.08046681019148, 55.14944118799408), (37.0..."
3,33622,38813,1,"[(37.08831009005705, 55.14946792727634), (37.0...","[(37.01696105787737, 55.21209316621927), (37.0..."
4,38412,38813,1,"[(37.04057941677159, 55.207692480896384), (37....","[(37.01696105787737, 55.21209316621927), (37.0..."


In [5]:
#считываем наименование всех регинов и прикручиваем к ним их координаты
#loc_names = pd.read_excel('names.xlsx')
#loc_names['cell_zid'] = loc_names['cell_zid'].map(int)
#loc_names['adm_zid'] = loc_names['adm_zid'].map(int)
#loc_names = loc_names.merge(right=s_df, how='inner', left_on='cell_zid', right_on='cell_zid')
loc_names = pd.read_csv('rebuilded_names.csv')
loc_names.head()

,Unnamed: 0,cell_zid,area_peresechenia_s_admzone_kv.km,adm_zid,adm_name,okrug_name,sub_ter
0,0,32909,0.407116,216,Роговское,Троицкий административный округ,Новая Москва
1,1,32910,1.003458,216,Роговское,Троицкий административный округ,Новая Москва
2,2,33261,0.233312,216,Роговское,Троицкий административный округ,Новая Москва
3,3,33262,1.868032,216,Роговское,Троицкий административный округ,Новая Москва
4,4,33263,2.500630,216,Роговское,Троицкий административный округ,Новая Москва


In [6]:
#считываем информацию о плотности проживающего, работающего и проходящего населения для каждого квадрата и прикручиваем туда координаты квадратов
c_locations = pd.read_csv("01_CLocation_July.csv")
c_locations = c_locations.merge(right=s_df, how='inner', left_on='zid', right_on='cell_zid')
c_locations.drop('cell_zid', axis=1, inplace=True)
#c_locations['coords'] = c_locations['coords'].apply(lambda x: (x[0][1],x[0][0] )) # оставляем только 1 координату
c_locations['lat'] = c_locations['coords'].map(lambda x: x[0][1]) # извлекаем широту
c_locations['lon'] = c_locations['coords'].map(lambda x: x[0][0]) # извлекаем долготу

c_locations.drop('coords', axis=1, inplace=True)


c_locations.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon
0,32909,4,0,0,0,55.140287,37.033512
1,32910,25,8,26,0,55.140316,37.041354
2,33261,2,0,2,0,55.144748,37.025618
3,33262,13,2,11,0,55.144778,37.033460
4,33263,10,0,3,0,55.144808,37.041302


In [7]:
adm_names = loc_names['adm_name'].drop_duplicates()
print("Кол-во уникальных райнов: ",adm_names.shape[0])

Кол-во уникальных райнов:  146


In [8]:
c_locations.loc[c_locations['zid'].isin(loc_names.loc[loc_names['adm_name'] == 'Северное Тушино']['cell_zid'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 9463 to 9764
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   zid                 50 non-null     int64  
 1   customers_cnt_home  50 non-null     int64  
 2   customers_cnt_job   50 non-null     int64  
 3   customers_cnt_day   50 non-null     int64  
 4   customers_cnt_move  50 non-null     int64  
 5   lat                 50 non-null     float64
 6   lon                 50 non-null     float64
dtypes: float64(2), int64(5)
memory usage: 3.1 KB


Сгенерируем псевдоданные за предыдущий месяц:

In [9]:
#Изменяем исходный датасет июля на рандомные % чтобы сгененрировать динамику плотности населения
c_locations_june = c_locations.copy()
c_locations_june['customers_cnt_home'] = c_locations_june['customers_cnt_home'].apply(lambda x: int(float(x) *( random.uniform(0.5, 1.7))))
c_locations_june['customers_cnt_job'] = c_locations_june['customers_cnt_job'].apply(lambda x: int(float(x) *( random.uniform(0.8, 1.1))))
c_locations_june['customers_cnt_day'] = c_locations_june['customers_cnt_day'].apply(lambda x: int(float(x) *( random.uniform(0.8, 1.1))))
c_locations_june['customers_cnt_move'] = c_locations_june['customers_cnt_move'].apply(lambda x: int(float(x) *( random.uniform(0.8, 1.1))))

c_locations_june.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon
0,32909,2,0,0,0,55.140287,37.033512
1,32910,13,6,26,0,55.140316,37.041354
2,33261,2,0,2,0,55.144748,37.025618
3,33262,13,1,11,0,55.144778,37.033460
4,33263,7,0,2,0,55.144808,37.041302


In [10]:
c_locations.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon
0,32909,4,0,0,0,55.140287,37.033512
1,32910,25,8,26,0,55.140316,37.041354
2,33261,2,0,2,0,55.144748,37.025618
3,33262,13,2,11,0,55.144778,37.033460
4,33263,10,0,3,0,55.144808,37.041302


In [75]:
delta_cnt_df = c_locations_june.copy()
delta_cnt_df = delta_cnt_df.merge(right=c_locations, how='inner', left_on='zid', right_on='zid')
delta_cnt_df.drop(['lat_y', 'lon_y'], axis=1, inplace=True)
delta_cnt_df.rename({'lat_x':'lat', 'lon_x':'lon'}, axis=1, inplace=True)

delta_cnt_df.head()

,zid,customers_cnt_home_x,customers_cnt_job_x,customers_cnt_day_x,customers_cnt_move_x,lat,lon,customers_cnt_home_y,customers_cnt_job_y,customers_cnt_day_y,customers_cnt_move_y
0,32909,2,0,0,0,55.140287,37.033512,4,0,0,0
1,32910,13,6,26,0,55.140316,37.041354,25,8,26,0
2,33261,2,0,2,0,55.144748,37.025618,2,0,2,0
3,33262,13,1,11,0,55.144778,37.033460,13,2,11,0
4,33263,7,0,2,0,55.144808,37.041302,10,0,3,0


In [76]:
#расчёт жельты изменения плотностей
for feature_type in ['home','job','day', 'move']:
    delta_cnt_df[f'customers_cnt_{feature_type}_y'] = delta_cnt_df[f'customers_cnt_{feature_type}_x'] - delta_cnt_df[f'customers_cnt_{feature_type}_y']
    delta_cnt_df.rename({f'customers_cnt_{feature_type}_y':f'customers_dlt_{feature_type}'},axis=1, inplace=True)
    delta_cnt_df.rename({f'customers_cnt_{feature_type}_x':f'customers_cnt_{feature_type}'},axis=1, inplace=True)

delta_cnt_df.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon,customers_dlt_home,customers_dlt_job,customers_dlt_day,customers_dlt_move
0,32909,2,0,0,0,55.140287,37.033512,-2,0,0,0
1,32910,13,6,26,0,55.140316,37.041354,-12,-2,0,0
2,33261,2,0,2,0,55.144748,37.025618,0,0,0,0
3,33262,13,1,11,0,55.144778,37.033460,0,-1,0,0
4,33263,7,0,2,0,55.144808,37.041302,-3,0,-1,0


In [31]:
#Настройка влияния весов на параметры плотности людей
alphas = {'home':1.0,'job':1.0,'day':1.0, 'move':1.0, 'passability':1.0}
alphas_dlt = {'home':1.0,'job':1.0,'day':1.0, 'move':1.0, 'passability':1.0}

In [41]:
print('уникальные точки "из дома": ',h_w_matrix.groupby(['home_zid']).size().reset_index().rename(columns={0:'count'}).shape[0])

уникальные точки "из дома":  9219


In [40]:
print('уникальные точки "до работы": ',h_w_matrix.groupby(['work_zid']).size().reset_index().rename(columns={0:'count'}).shape[0])

уникальные точки "до работы":  9016


In [52]:
delta_cnt_df['logistic'] = 0
delta_cnt_df['logistic'] = delta_cnt_df['zid'].apply(lambda x: sum(h_w_matrix.loc[h_w_matrix['work_zid'] == x]['customers_cnt'].values))


In [77]:
delta_cnt_df.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon,customers_dlt_home,customers_dlt_job,customers_dlt_day,customers_dlt_move
0,32909,2,0,0,0,55.140287,37.033512,-2,0,0,0
1,32910,13,6,26,0,55.140316,37.041354,-12,-2,0,0
2,33261,2,0,2,0,55.144748,37.025618,0,0,0,0
3,33262,13,1,11,0,55.144778,37.033460,0,-1,0,0
4,33263,7,0,2,0,55.144808,37.041302,-3,0,-1,0


In [97]:
mfc_df = pd.read_excel('mos_coords.xlsx')
mfc_df['geodata_center'] = mfc_df['geodata_center'].apply(lambda x: (json.loads(x)['coordinates'][1],json.loads(x)['coordinates'][0])  )
mfc_df['lon'] = mfc_df['geodata_center'].apply(lambda x: x[0])
mfc_df['lat'] = mfc_df['geodata_center'].apply(lambda x: x[1])
mfc_df.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   object_category_Id           0 non-null      float64
 1   CommonName                   139 non-null    object 
 2   FullName                     139 non-null    object 
 3   ShortName                    139 non-null    object 
 4   INN                          139 non-null    int64  
 5   KPP                          139 non-null    int64  
 6   OGRN                         0 non-null      float64
 7   AdmArea                      139 non-null    object 
 8   District                     139 non-null    object 
 9   Address                      139 non-null    object 
 10  ChiefName                    139 non-null    object 
 11  ChiefPosition                139 non-null    object 
 12  PublicPhone                  139 non-null    object 
 13  WorkingHours        

In [116]:
def measure(lat1, lon1, lat2, lon2):  # generally used geo measurement function
    #PI = 3.141596
    R = 6378.137 # Radius of earth in KM
    dLat =lat2 - lat1 # lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon =lon2 - lon1# lon2 * math.pi / 180 - lon1 * math.pi / 180
    #a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * math.cos(lat2 * math.pi / 180) *  math.sin(dLon/2) * math.sin(dLon/2)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1) * math.cos(lat2) *  math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d * 1000; # meters

In [ ]:
tqdm.pandas(desc="find mfc progress!")

wellington = (-41.32, 174.81)
salamanca = (40.96, -5.50)


delta_cnt_df['nearest_mfc'] = 0
delta_cnt_df['nearest_mfc'] = delta_cnt_df['zid'].progress_apply(
    lambda x: geopy.distance.distance(mfc_df.loc[mfc_df['geodata_center'].apply(
        lambda y: geopy.distance.distance((y[0],y[1]),
                          (delta_cnt_df.loc[delta_cnt_df['zid']==x]['lat'].values[0],
                           delta_cnt_df.loc[delta_cnt_df['zid']==x]['lon'].values[0])
                                    ).m
    ).idxmin()]['geodata_center'], (delta_cnt_df.loc[delta_cnt_df['zid']==x]['lat'].values[0],
                           delta_cnt_df.loc[delta_cnt_df['zid']==x]['lon'].values[0])).m)
    
#mfc_df.loc[mfc_df['lat'].apply(lambda y: abs(y -delta_cnt_df.loc[delta_cnt_df['zid']==x]['lat'].values[0])).idxmin()&&mfc_df.loc[mfc_df['lon'].apply(lambda y: abs(y -delta_cnt_df.loc[delta_cnt_df['zid']==x]['lon'].values[0])).idxmin()]['lon']
#measure(delta_cnt_df.loc[delta_cnt_df['zid']==x]['lat'].values[0],delta_cnt_df.loc[delta_cnt_df['zid']==x]['lon'].values[0],

delta_cnt_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
find mfc progress!:  76%|██████████████████████████████████████████████████████████████████████▋                      | 7790/10240 [24:44<08:09,  5.01it/s]

In [ ]:
print('max:', delta_cnt_df['nearest_mfc'].max(), ' min:', delta_cnt_df['nearest_mfc'].min() )

In [125]:
mfc_df['geodata_center'].values[0][0]

55.785871804304